In [ ]:
%run ../lib/kops.py
%run ../lib/e2e.py
%run ../lib/state.py

state = load_state()
state

In [ ]:
# For GCE: zones = ["us-central1-f"]
zones = ["us-east-2a"]

In [ ]:
# We're going to update from 1.16 to 1.17
kubernetes_version = get_kubernetes_version("stable-1.16")
update_state(state, {"kops_create_cluster_spec": { "kubernetes_version": kubernetes_version }})
state

In [ ]:
update_state(state, {"kops_create_cluster_spec": { "networking": "cilium" }})
state

In [ ]:
if not "project" in state:
    p = os.environ.get("PROJECT_ID")
    if not p:
        p = gcloud.current_project()
    update_state(state, { "project": p })

In [ ]:
if not "kops_version" in state:
    update_state(state, { "kops_version": get_kops_ci_latest() })

In [ ]:
create_cluster_spec_defaults = { "name": "foo.k8s.local", "project": state["project"], "zones": zones, "cluster": { "spec": { "nodePortAccess": "0.0.0.0/0" } } }
update_state(state, {"kops_create_cluster_spec": create_cluster_spec_defaults})

In [ ]:
if not "results" in state:
    update_state(state, {"results": [] })
update_state(state, {"schema": "e2e.kops.k8s.io/v1"})

In [ ]:
save_state(state)
state

In [ ]:
kops = download_kops(state["kops_version"])
kops.version()

In [ ]:
cluster_name = state["kops_create_cluster_spec"]["name"]

In [ ]:
c = kops.create_cluster(state["kops_create_cluster_spec"])

In [ ]:
c = kops.cluster(cluster_name)

In [ ]:
c

In [ ]:
update_state(state, { "kops_spec": c.objects()})
save_state(state)

In [ ]:
c.update()

In [ ]:
c.wait()

# Here is where we should run tests
# fork_tests()

In [ ]:
new_version = get_kubernetes_version("stable-1.17")
c.set("spec.kubernetesVersion", new_version)

In [ ]:
c.preview_upgrade()
c.upgrade()


In [ ]:
c.update()

In [ ]:
c.rolling_update()

In [ ]:
c.wait()

# Here is where we should run more tests
# fork_tests()

# Cleanup

In [ ]:
c.delete()